# Neural Network / 신경망

An nn.Module contains layers, and a method forward(input)that returns the output.
/ nn.Module 은 계층(layer)과 output 을 반환하는 forward(input) 메서드를 포함하고 있습니다.



---

![대체 텍스트](https://pytorch.org/tutorials/_images/mnist.png)


> It is a simple feed-forward network. It takes the input, feeds it through several layers one after the other, and then finally gives the output.

A typical training procedure for a neural network is as follows:

* Define the neural network that has some learnable parameters (or weights)
* Iterate over a dataset of inputs
* Process input through the network
* Compute the loss (how far is the output from being correct)
* Propagate gradients back into the network’s parameters
* Update the weights of the network, typically using a simple update rule: weight = weight - learning_rate * gradient



---

>이는 간단한 순전파 네트워크(Feed-forward network)입니다. 입력(input)을 받아 여러 계층에 차례로 전달한 후, 최종 출력(output)을 제공합니다.

신경망의 일반적인 학습 과정은 다음과 같습니다:

* 학습 가능한 매개변수(또는 가중치(weight))를 갖는 신경망을 정의합니다.

* 데이터셋(dataset) 입력을 반복합니다.

* 입력을 신경망에서 전파(process)합니다.

* 손실(loss; 출력이 정답으로부터 얼마나 떨어져있는지)을 계산합니다.

* 변화도(gradient)를 신경망의 매개변수들에 역으로 전파합니다.

* 신경망의 가중치를 갱신합니다. 일반적으로 다음과 같은 간단한 규칙을 사용합니다: 가중치(wiehgt) = 가중치(weight) - 학습율(learning rate) * 변화도(gradient)


# Define the Network / 신경망 정의

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):           
        # Max pooling over a (2, 2) window
        # Forward 함수 정의하면, (변화도 계산하는) backward 함수는 autograd를 사용하여 자동으로 정의

        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


The learnable parameters of a model are returned by net.parameters() / 모델의 학습 가능한 매개변수들은 net.parameters() 에 의해 반환됩니다.

In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


![대체 텍스트](https://www.pyimagesearch.com/wp-content/uploads/2016/06/lenet_architecture-768x226.png)

Note: expected input size of this net (LeNet) is 32x32. To use this net on the MNIST dataset, please resize the images from the dataset to 32x32.



---

Note: 이 신경망(LeNet)의 예상되는 입력 크기는 32x32입니다. 이 신경망에 MNIST 데이터셋을 사용하기 위해서는, 데이터셋의 이미지 크기를 32x32로 변경해야 합니다.


In [3]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0221, -0.1019, -0.0370,  0.0806,  0.1099, -0.0173, -0.0500, -0.0744,
         -0.0168, -0.0533]], grad_fn=<AddmmBackward>)


torch.nn only supports mini-batches. The entire torch.nn package only supports inputs that are a mini-batch of samples, and not a single sample.

For example, nn.Conv2d will take in a 4D Tensor of nSamples x nChannels x Height x Width.

If you have a single sample, just use input.unsqueeze(0) to add a fake batch dimension.

Before proceeding further, let’s recap all the classes you’ve seen so far.

Recap:
torch.Tensor - A multi-dimensional array with support for autograd operations like backward(). Also holds the gradient w.r.t. the tensor.

nn.Module - Neural network module. Convenient way of encapsulating parameters, with helpers for moving them to GPU, exporting, loading, etc.

nn.Parameter - A kind of Tensor, that is automatically registered as a parameter when assigned as an attribute to a Module.

autograd.Function - Implements forward and backward definitions of an autograd operation. Every Tensor operation creates at least a single Function node that connects to functions that created a Tensor and encodes its history.

At this point, we covered:
Defining a neural network
Processing inputs and calling backward

Still Left:
Computing the loss
Updating the weights of the network

---

torch.nn 은 미니-배치(mini-batch)만 지원합니다. torch.nn 패키지 전체는 하나의 샘플이 아닌, 샘플들의 미니-배치만을 입력으로 받습니다.

예를 들어, nnConv2D 는 nSamples x nChannels x Height x Width 의 4차원 Tensor를 입력으로 합니다.

만약 하나의 샘플만 있다면, input.unsqueeze(0) 을 사용해서 가짜 차원을 추가합니다.

계속 진행하기 전에, 지금까지 살펴봤던 것들을 다시 한번 요약해보겠습니다.


요약:

torch.Tensor - backward() 같은 autograd 연산을 지원하는 다차원 배열 입니다. 또한 tensor에 대한 변화도(gradient)를 갖고 있습니다.

nn.Module - 신경망 모듈. 매개변수를 캡슐화(encapsulation)하는 간편한 방법 으로, GPU로 이동, 내보내기(exporting), 불러오기(loading) 등의 작업을 위한 헬퍼(helper)를 제공합니다.

nn.Parameter - Tensor의 한 종류로, Module 에 속성으로 할당될 때 자동으로 매개변수로 등록 됩니다.

autograd.Function - autograd 연산의 전방향과 역방향 정의 를 구현합니다. 모든 Tensor 연산은 하나 이상의 Function 노드를 생성하며, 각 노드는 Tensor 를 생성하고 이력(history)을 부호화 하는 함수들과 연결하고 있습니다.

지금까지 우리가 다룬 내용은 다음과 같습니다:
신경망을 정의하는 것

입력을 처리하고 backward 를 호출하는 것

더 살펴볼 내용들은 다음과 같습니다:
손실을 계산하는 것

신경망의 가중치를 갱신하는 것


# Loss Function / 손실 함수

> A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.

> There are several different loss functions under the nn package . A simple loss is: nn.MSELoss which computes the mean-squared error between the input and the target.



---

> 손실 함수는 (output, target)을 한 쌍(pair)의 입력으로 받아, 출력(output)이 정답(target)으로부터 얼마나 멀리 떨어져있는지 추정하는 값을 계산합니다.

> nn 패키지에는 여러가지의 손실 함수들 이 존재합니다. 간단한 손실 함수로는 출력과 대상간의 평균제곱오차(mean-squared error)를 계산하는 nn.MSEloss 가 있습니다.


In [6]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.1501, grad_fn=<MseLossBackward>)


> Now, if you follow loss in the backward direction, using its .grad_fn attribute, you will see a graph of computations that looks like this:



---

> 이제 .grad_fn 속성을 사용하여 loss 를 역방향에서 따라가다보면, 이러한 모습의 연산 그래프를 볼 수 있습니다:



> So, when we call loss.backward(), the whole graph is differentiated w.r.t. the loss, and all Tensors in the graph that has requires_grad=True will have their .grad Tensor accumulated with the gradient.



---



> 따라서 loss.backward() 를 실행할 때, 전체 그래프는 손실(loss)에 대하여 미분되며, 그래프 내의 requires_grad=True 인 모든 Tensor는 변화도(gradient)가 누적된 .grad Tensor를 갖게 됩니다.

In [7]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

# Backprop / 역전파

![대체 텍스트](https://miro.medium.com/max/1678/1*CxdjKFrE-Vww0KmI-3Z5sA.png)

> To backpropagate the error all we have to do is to loss.backward(). You need to clear the existing gradients though, else gradients will be accumulated to existing gradients.



---

> 오차(error)를 역전파하기 위해서는 loss.backward() 만 해주면 됩니다. 기존 변화도를 없애는 작업이 필요한데, 그렇지 않으면 변화도가 기존의 것에 누적되기 때문입니다.





> Now we shall call loss.backward(), and have a look at conv1’s bias gradients before and after the backward.

---



> 이제 loss.backward() 를 호출하여 역전파 전과 후에 conv1의 bias gradient를 살펴보겠습니다.

In [8]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0027, -0.0131, -0.0141, -0.0238,  0.0044,  0.0293])


# Update the Weights / 가중치 갱신

> The simplest update rule used in practice is the Stochastic Gradient Descent (SGD):

> weight = weight - learning_rate * gradient

In [0]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [0]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update